In [1]:
# Setting up the Coordinate Reference Systems up front in the necessary format.
crs_degree = {'init': 'epsg:4326'} # CGS_WGS_1984 (what the GPS uses)

# --- Paths

# Root path of Fremont Dropbox
import os
import sys
# We let this notebook to know where to look for fremontdropbox module
module_path = os.path.abspath(os.path.join('../..'))
if module_path not in sys.path:
    sys.path.append(module_path)
    
from fremontdropbox import get_dropbox_location
# Root path of the Dropbox business account



dbx = get_dropbox_location()

# Temporary! Location of the folder where the restructuring is currently happening
data_path = dbx + '/Private Structured data collection'

aux_files = data_path+'/Data processing/Auxiliary files'

# Processing output path
output_path = aux_files + '/OD demand'

In [2]:
# Read more about GeoPandas data structures here: http://geopandas.org/data_structures.html
from geopandas import GeoDataFrame
import geopandas as gpd # To work with spatial data in a DataFrame
from shapely.geometry import Point, LineString # To create line geometries that can be used in a GeoDataFrame
from keplergl import KeplerGl
import pandas as pd

# packages for network manipulation and visualization
import csv
from operator import itemgetter
import networkx as nx
import osmnx as ox
import matplotlib.pyplot as plt
import matplotlib.lines as mlines
#import cartopy.crs as ccrs
from IPython.display import Image
#from mpl_toolkits.basemap import Basemap as Basemap #Installation failed
from networkx.algorithms import community #This part of networkx, for community detection, needs to be imported separately.

### Load the Aimsun network using:
- Geopandas (to read and convert the files) 
- Networkx (to convert the data to a network) 
- Visualize the Geopandas network shapefile in Kepler.gl

In [5]:
#centroid_connections = GeoDataFrame.from_file(data_path + "/Data processing/Raw/Network/Aimsun/centroid_connections.shp")
#centroids = GeoDataFrame.from_file(data_path + "/Data processing/Raw/Network/Aimsun/centroids.shp")
#detectors = GeoDataFrame.from_file(data_path + "/Data processing/Raw/Network/Aimsun/detectors.shp")
#meterings = GeoDataFrame.from_file(data_path + "/Data processing/Raw/Network/Aimsun/meterings.shp")
nodes = GeoDataFrame.from_file(data_path + "/Data processing/Raw/Network/Aimsun/nodes.shp")
#polygons = GeoDataFrame.from_file(data_path + "/Data processing/Raw/Network/Aimsun/polygons.shp")
sections = GeoDataFrame.from_file(data_path + "/Data processing/Raw/Network/Aimsun/sections.shp")
#sectionsGeo = GeoDataFrame.from_file(data_path + "/Data processing/Raw/Network/Aimsun/sectionsGeo.shp")


In [6]:
Aimsun_nodes = nodes.to_crs(epsg=4326)
Aimsun_sections = sections.to_crs(epsg=4326)

In [7]:
Aimsun_sections

,id,eid,name,nb_lanes,speed,capacity,rd_type,func_class,fnode,tnode,geometry
0,242.0,242,None,1,120.0,2100.0,175.0,1,9845.0,9923.0,"LINESTRING (-121.92244 37.49593, -121.92242 37..."
1,243.0,243,None,3,104.0,6300.0,175.0,1,9852.0,9848.0,"LINESTRING (-121.92313 37.49526, -121.92173 37..."
2,244.0,244,None,3,104.0,6300.0,175.0,1,9850.0,9852.0,"LINESTRING (-121.92352 37.49561, -121.92313 37..."
3,246.0,246,Geyser Court,1,50.0,700.0,179.0,5,9868.0,NaN,"LINESTRING (-121.91449 37.50455, -121.91477 37..."
4,247.0,247,Geyser Court,1,50.0,700.0,179.0,5,NaN,9868.0,"LINESTRING (-121.91572 37.50422, -121.91560 37..."
...,...,...,...,...,...,...,...,...,...,...,...
7316,62345.0,62345,None,1,50.0,500.0,185.0,5,NaN,62346.0,"LINESTRING (-121.94146 37.51175, -121.94142 37..."
7317,62683.0,62683,None,2,50.0,1800.0,177.0,3,62687.0,62685.0,"LINESTRING (-121.91671 37.52686, -121.91722 37..."
7318,62684.0,62684,None,2,50.0,1800.0,177.0,3,62691.0,62687.0,"LINESTRING (-121.91524 37.52747, -121.91549 37..."
7319,62846.0,62846,None,1,50.0,900.0,177.0,3,62848.0,53511.0,"LINESTRING (-121.91368 37.52917, -121.91371 37..."


In [9]:
map_1 = KeplerGl(height=1000)
map_1.add_data(data=Aimsun_sections, name = "sections")
map_1.add_data(data=Aimsun_nodes, name = "nodes")
map_1



User Guide: https://github.com/keplergl/kepler.gl/blob/master/docs/keplergl-jupyter/user-guide.md


KeplerGl(data={'sections': {'index': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20…

In [10]:
# Loading the Aimsun network in networkx
Nodes = nx.read_shp(data_path + "/Data processing/Raw/Network/Aimsun/nodes.shp")
Edges = nx.read_shp(data_path + "/Data processing/Raw/Network/Aimsun/sections.shp")
#Point of Interests scraped from Google API
POIs = pd.read_csv(data_path + "/Data processing/Raw/Network/KPIs/locations_crawl.csv")
POIs['geometry'] = [Point(xy) for xy in zip(POIs['Long'], POIs['Lat'])]
crs = {'init': 'epsg:4326'}
# Creating a Geographic data frame for Point of Interests
POIs_gdf = gpd.GeoDataFrame(POIs, crs=crs)

/Users/LiJiayi/anaconda3/envs/KPI_calc/lib/python3.8/site-packages/pyproj/crs/crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))


In [11]:
POIs_gdf.head(5)

,Unnamed: 0,Names,Types,Lat,Long,geometry
0,0,Fremont,"['locality', 'political']",37.548540,-121.988583,POINT (-121.98858 37.54854)
1,1,ProCreativeWriters,"['point_of_interest', 'establishment']",37.504377,-121.964423,POINT (-121.96442 37.50438)
2,2,Baylands,"['neighborhood', 'political']",37.485034,-121.964375,POINT (-121.96437 37.48503)
3,3,Fremont,"['locality', 'political']",37.548540,-121.988583,POINT (-121.98858 37.54854)
4,4,Mission Sierra Taekwon-Do,"['health', 'point_of_interest', 'establishment']",37.532586,-121.922044,POINT (-121.92204 37.53259)


In [12]:
#creating the networkx graph
G = nx.Graph()
G.add_nodes_from(Nodes.nodes)
G.add_edges_from(Edges.edges)

In [13]:
print("the amount of nodes in the graph")
print(len(list(G.nodes(data=True))))
print("the amount of edges in the graph")
print(len(list(G.edges(data=True))))

the amount of nodes in the graph
16911
the amount of edges in the graph
7320


At the time, all nodes and edges are un-weighted: the weights are not assigned yet

In [14]:
def node_getter(index): 
    """
    retrieve the lat and lon of a node at specific index
    """
    return list(G.nodes(data=True))[index][0]
def edge_getter(index):
    """
    retrieve the starting node and endinge node of of an edge at specific index
    """
    u, v, w = list(G.edges(data=True))[index]
    return (u, v, w)

In [12]:
#Applying BFS in networkx G
list(nx.bfs_edges(G, source=node_getter(1), depth_limit=10))

[((595317.6669960958, 4150223.117825553),
  (595192.2991812585, 4150360.529504859)),
 ((595317.6669960958, 4150223.117825553),
  (595530.1292144678, 4149942.1818685057)),
 ((595192.2991812585, 4150360.529504859),
  (595157.244298812, 4150398.3882524366))]

Assign a preliminary weight 1 to each edge

In [13]:
for i in range(len(G.edges(data=True))):
    u, v, w = edge_getter(i)
    G[u][v]['weight'] = 1

KeyboardInterrupt: 

Transform the 

In [ ]:
edges_ref = {}
for i in range(len(G.edges(data=True))):
    u, v, w = edge_getter(i)
    name = '{}_{}'.format(u, v)
    e2_dict = {}
    e2_dict['st_node'] = u
    e2_dict['en_node'] = v
    e2_dict['weight'] = 1
    edges_ref[name] = e2_dict


### Finding the nearest network node for each POIs (abandoned section)

In [ ]:
import geopandas as gpd
import numpy as np
import pandas as pd

from scipy.spatial import cKDTree
from shapely.geometry import Point

gpd1 = gpd.GeoDataFrame([['John', 1, Point(1, 1)], ['Smith', 1, Point(2, 2)],
                         ['Soap', 1, Point(0, 2)]],
                        columns=['Name', 'ID', 'geometry'])
gpd2 = gpd.GeoDataFrame([['Work', Point(0, 1.1)], ['Shops', Point(2.5, 2)],
                         ['Home', Point(1, 1.1)]],
                        columns=['Place', 'geometry'])

def ckdnearest(gdA, gdB):
    nA = np.array(list(zip(gdA.geometry.x, gdA.geometry.y)) )
    nB = np.array(list(zip(gdB.geometry.x, gdB.geometry.y)) )
    btree = cKDTree(nB)
    dist, idx = btree.query(nA, k=1)
    gdf = pd.concat(
        [gdA.reset_index(drop=True), gdB.loc[idx, gdB.columns != 'geometry'].reset_index(drop=True),
         pd.Series(dist, name='dist')], axis=1)
    return gdf

ckdnearest(gpd1, gpd2)

## Using Pandana package to perform the accessibility calculation

In [15]:
import pandana as pdna
from pandana.loaders import osm
%matplotlib inline
from descartes import PolygonPatch
import random
import warnings
warnings.filterwarnings('ignore')

preprocess the dataset before dumping the network into pandana

In [16]:
Aimsun_nodes['x'] = Aimsun_nodes['geometry'].x
Aimsun_nodes['y'] = Aimsun_nodes['geometry'].y

In [17]:
Aimsun_nodes = Aimsun_nodes.rename(columns={'id': "id_node"})
Aimsun_nodes['id_node'] = Aimsun_nodes['id_node'].astype(int)
Aimsun_nodes.index = Aimsun_nodes['id_node']
Aimsun_nodes_gdf = Aimsun_nodes[['id_node', 'x', 'y']]


In [18]:
Aimsun_nodes_gdf.head(5)

,id_node,x,y
id_node,,,
9845,9845,-121.922491,37.495935
9848,9848,-121.921727,37.494013
9850,9850,-121.923525,37.495613
9852,9852,-121.923128,37.495265
9854,9854,-121.934705,37.505858


In [19]:
Aimsun_sections.index = range(len(Aimsun_sections))
Aimsun_sections_gdf = Aimsun_sections[['id', 'fnode', 'tnode', 'capacity']]
Aimsun_sections_gdf['id'] = Aimsun_sections_gdf['id'].fillna(0.0).astype(int)

In [20]:
Aimsun_sections['fnode'].fillna(9845, inplace=True)
Aimsun_sections['tnode'].fillna(9845, inplace=True)
Aimsun_sections.index = range(len(Aimsun_sections))
Aimsun_sections_gdf = Aimsun_sections[['id', 'eid', 'fnode', 'tnode', 'capacity']]
Aimsun_sections_gdf['id'] = Aimsun_sections_gdf['id'].fillna(0.0).astype(int)

Aimsun_sections_gdf['tnode'] = Aimsun_sections_gdf['tnode'].astype(int)
Aimsun_sections_gdf['fnode'] = Aimsun_sections_gdf['fnode'].astype(int)
Aimsun_sections_gdf.id = range(len(Aimsun_sections_gdf))

In [21]:
Aimsun_sections

,id,eid,name,nb_lanes,speed,capacity,rd_type,func_class,fnode,tnode,geometry
0,242.0,242,None,1,120.0,2100.0,175.0,1,9845.0,9923.0,"LINESTRING (-121.92244 37.49593, -121.92242 37..."
1,243.0,243,None,3,104.0,6300.0,175.0,1,9852.0,9848.0,"LINESTRING (-121.92313 37.49526, -121.92173 37..."
2,244.0,244,None,3,104.0,6300.0,175.0,1,9850.0,9852.0,"LINESTRING (-121.92352 37.49561, -121.92313 37..."
3,246.0,246,Geyser Court,1,50.0,700.0,179.0,5,9868.0,9845.0,"LINESTRING (-121.91449 37.50455, -121.91477 37..."
4,247.0,247,Geyser Court,1,50.0,700.0,179.0,5,9845.0,9868.0,"LINESTRING (-121.91572 37.50422, -121.91560 37..."
...,...,...,...,...,...,...,...,...,...,...,...
7316,62345.0,62345,None,1,50.0,500.0,185.0,5,9845.0,62346.0,"LINESTRING (-121.94146 37.51175, -121.94142 37..."
7317,62683.0,62683,None,2,50.0,1800.0,177.0,3,62687.0,62685.0,"LINESTRING (-121.91671 37.52686, -121.91722 37..."
7318,62684.0,62684,None,2,50.0,1800.0,177.0,3,62691.0,62687.0,"LINESTRING (-121.91524 37.52747, -121.91549 37..."
7319,62846.0,62846,None,1,50.0,900.0,177.0,3,62848.0,53511.0,"LINESTRING (-121.91368 37.52917, -121.91371 37..."


In [22]:
Aimsun_sections_gdf.head(5)

,id,eid,fnode,tnode,capacity
0,0,242,9845,9923,2100.0
1,1,243,9852,9848,6300.0
2,2,244,9850,9852,6300.0
3,3,246,9868,9845,700.0
4,4,247,9845,9868,700.0


In [23]:
id_keys = range(len(Aimsun_nodes_gdf))
id_values = Aimsun_nodes_gdf.id_node
id_dictionary = dict(zip(id_keys, id_values))

x_values = Aimsun_nodes_gdf.x
x_dictionary = dict(zip(id_keys, x_values))

y_values = Aimsun_nodes_gdf.y
y_dictionary = dict(zip(id_keys, y_values))

edge_keys = range(len(Aimsun_sections_gdf))
edge_values = Aimsun_sections.eid.astype(int)
edge_dictionary = dict(zip(edge_keys, edge_values))

from_values = Aimsun_sections_gdf.fnode.fillna(0.0).astype(int)
to_values = Aimsun_sections_gdf.tnode.fillna(0.0).astype(int)

from_dict = dict(zip(edge_keys, from_values))
to_dict = dict(zip(edge_keys, to_values))

weight_values = Aimsun_sections_gdf.capacity
weight_dict = dict(zip(edge_keys, weight_values))

In [24]:
#check to ensure that all the section ends are network nodes
node_set = set(Aimsun_nodes_gdf.id_node)
for node in Aimsun_sections.fnode.fillna(0.0).astype(int):
    if node not in node_set:
        print(node)

In [ ]:
Aimsun_edge_dict = {
 'id': edge_dictionary,
 'id_node_source': from_dict,
 'id_node_target': to_dict,
 'distance': weight_dict}

node_dict = {
 'id_node': id_dictionary,
 'x': x_dictionary,
 'y': y_dictionary}

# read dictionary into dataframe
edges_topo = pd.DataFrame.from_dict(Aimsun_edge_dict)
nodes_gdf = pd.DataFrame.from_dict(node_dict)
nodes_gdf.index = nodes_gdf['id_node']



## Loading the edge weights into the Pandana network sections

In [42]:
vehSectTraj_temp

,Unnamed: 0,did,oid,ent,exitTime,travelTime,delayTime
sectionId,,,,,,,
242,233,27989,10,24,50481.9,12.94600,7.130020
243,15,27989,1,16,50626.1,5.19072,0.000000
244,14,27989,1,15,50620.9,1.47734,0.037734
248,9462,27989,367,3,50654.4,12.32940,8.871620
249,896850,27989,31941,68,64888.0,4.72323,1.716150
...,...,...,...,...,...,...,...
56184,1099,27989,43,8,50560.1,7.16969,0.000000
56532,1138203,27989,41443,30,66486.2,5.67516,0.000000
56553,1098,27989,43,7,50551.5,5.64885,0.000000


In [44]:
vehSectTraj = pd.read_csv(data_path + '/Aimsun/Outputs/vehSectTrajectory.csv')
vehSectTraj = vehSectTraj[vehSectTraj.notna()]

In [45]:
vehSectTraj_temp = vehSectTraj.groupby("sectionId").first()

In [46]:
edge_weighted = pd.merge(edges_topo,
                           vehSectTraj_temp,
                           left_on='id',
                           right_on='sectionId',
                           how='left',
                           sort=True)

,Unnamed: 0,did,oid,ent,sectionId,exitTime,travelTime,delayTime
0,0,27989,1,1,2873,50420.3,19.334500,0.000000
1,1,27989,1,2,2874,50433.3,12.741000,0.065405
2,2,27989,1,3,2880,50435.1,1.844950,0.000000
3,3,27989,1,4,7155,50444.2,9.116290,0.000000
4,4,27989,1,5,9212,50447.5,3.239390,0.000000
...,...,...,...,...,...,...,...,...
1539564,1539564,27989,74337,1,2682,72898.3,4.328070,0.000000
1539565,1539565,27989,74342,1,1249,72899.5,4.406190,1.136560
1539566,1539566,27989,74343,1,628,72897.9,2.725480,0.416010
1539567,1539567,27989,74356,1,35861,72900.0,2.006960,0.000000


In [47]:
net = pdna.Network(node_x = nodes_gdf["x"],
                   node_y = nodes_gdf["y"],
                   edge_from = edge_weighted["id_node_source"], 
                   edge_to = edge_weighted["id_node_target"],
                   edge_weights = edge_weighted[["distance"]])

In [48]:
#scraped Point of Interests in Fremont Area
POIs_gdf.head()

,Unnamed: 0,Names,Types,Lat,Long,geometry
0,0,Fremont,"['locality', 'political']",37.548540,-121.988583,POINT (-121.98858 37.54854)
1,1,ProCreativeWriters,"['point_of_interest', 'establishment']",37.504377,-121.964423,POINT (-121.96442 37.50438)
2,2,Baylands,"['neighborhood', 'political']",37.485034,-121.964375,POINT (-121.96437 37.48503)
3,3,Fremont,"['locality', 'political']",37.548540,-121.988583,POINT (-121.98858 37.54854)
4,4,Mission Sierra Taekwon-Do,"['health', 'point_of_interest', 'establishment']",37.532586,-121.922044,POINT (-121.92204 37.53259)


In [49]:
bounding_box = Aimsun_sections.unary_union.envelope
tem_df = gpd.GeoDataFrame(gpd.GeoSeries(bounding_box), columns=['geometry'])
intersections = gpd.overlay(POIs_gdf, tem_df, how='intersection')

In [50]:
#save the scraped Point of Interests to shapefile
intersections.to_file(driver = 'ESRI Shapefile', filename= data_path + "/Data processing/Raw/Network/Aimsun/POI.shp")

In [51]:
map_2 = KeplerGl(height=1000)
map_2.add_data(data=intersections, name = "Point Of Interests")
map_2.add_data(data=Aimsun_sections, name = "sections")
map_2.add_data(data=Aimsun_nodes, name = "nodes")
map_2




User Guide: https://github.com/keplergl/kepler.gl/blob/master/docs/keplergl-jupyter/user-guide.md


KeplerGl(data={'Point Of Interests': {'index': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, …

## Assign each POI to corresponding nearest Aimsun node

In [52]:
#get_node_ids uses the KDTree from scipy
near_ids = net.get_node_ids(intersections['Long'],
                            intersections['Lat'])

# Set the response as a new column on the POI reference df
intersections['nearest_node_id'] = near_ids

In [53]:
nodes_gdf.index = range(len(nodes_gdf))

In [54]:
# Create a merged dataframe that holds the node data (esp. x and y values)
# that relate to each nearest neighbor of each POI
nearest_to_pois = pd.merge(intersections,
                           nodes_gdf,
                           left_on='nearest_node_id',
                           right_on='id_node',
                           how='left',
                           sort=False,
                           suffixes=['_from', '_to'])

In [55]:
#showing how each POI is matched to the nearest node id
nearest_to_pois.head(5)

,Unnamed: 0,Names,Types,Lat,Long,geometry,nearest_node_id,id_node,x,y
0,4,Mission Sierra Taekwon-Do,"['health', 'point_of_interest', 'establishment']",37.532586,-121.922044,POINT (-121.92204 37.53259),26626,26626,-121.921920,37.532924
1,5,Hoard Siu PC,"['doctor', 'health', 'point_of_interest', 'est...",37.532591,-121.922271,POINT (-121.92227 37.53259),26645,26645,-121.922656,37.532504
2,6,Tavares Realty,"['real_estate_agency', 'point_of_interest', 'e...",37.533002,-121.922212,POINT (-121.92221 37.53300),23422,23422,-121.922149,37.533130
3,7,Unifier Learning Academy,"['point_of_interest', 'establishment']",37.532530,-121.922021,POINT (-121.92202 37.53253),26626,26626,-121.921920,37.532924
4,8,Oroysom Village,"['point_of_interest', 'establishment']",37.533912,-121.923178,POINT (-121.92318 37.53391),16578,16578,-121.923020,37.534326


In [56]:
POI_matching_lines = []
for row_id, row in nearest_to_pois.iterrows():
    linestr = LineString([(row['geometry'].x, row['geometry'].y),
                          (row['x'], row['y'])]).buffer(0.000001)
    POI_matching_lines.append(linestr)
    

In [57]:
POI_matching_lines_gdf = pd.DataFrame({'geometry':POI_matching_lines})
POI_matching_lines_gdf = gpd.GeoDataFrame(POI_matching_lines_gdf, crs=crs)

In [58]:
POI_matching_lines_gdf.head(5)

,geometry
0,"POLYGON ((-121.92192 37.53292, -121.92192 37.5..."
1,"POLYGON ((-121.92266 37.53250, -121.92266 37.5..."
2,"POLYGON ((-121.92215 37.53313, -121.92215 37.5..."
3,"POLYGON ((-121.92192 37.53292, -121.92192 37.5..."
4,"POLYGON ((-121.92302 37.53433, -121.92302 37.5..."


In [59]:
map_4 = KeplerGl(height=1000)
map_4.add_data(data=POI_matching_lines_gdf, name = "POI matching nodes")
map_4.add_data(data=intersections, name = "Point Of Interests")
map_4.add_data(data=Aimsun_sections, name = "sections")
map_4.add_data(data=Aimsun_nodes, name = "nodes")

map_4

User Guide: https://github.com/keplergl/kepler.gl/blob/master/docs/keplergl-jupyter/user-guide.md


KeplerGl(data={'POI matching nodes': {'index': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, …

In [77]:
net.set_pois("restaurants", 3000, 20, intersections['Long'], intersections['Lat'])

In [78]:
access = net.nearest_pois(3000, "restaurants", num_pois=10)

In [86]:
nodes_gdf

,id_node,x,y
0,9845,-121.922491,37.495935
1,9848,-121.921727,37.494013
2,9850,-121.923525,37.495613
3,9852,-121.923128,37.495265
4,9854,-121.934705,37.505858
...,...,...,...
2635,62955,-121.944719,37.547683
2636,62956,-121.913275,37.510602
2637,62957,-121.913258,37.510625
2638,62958,-121.930401,37.537421


In [84]:
POIs = pd.read_csv(data_path + "/Data processing/Raw/Network/KPIs/locations_crawl.csv")
POIs['geometry'] = [Point(xy) for xy in zip(POIs['Long'], POIs['Lat'])]
crs = {'init': 'epsg:4326'}
# Creating a Geographic data frame for Point of Interests
POIs_gdf = gpd.GeoDataFrame(POIs, crs=crs)

## To be done: accessibility visualization Measures

In [100]:
n = 1
temp_access = access[[n]]
nodes_gdf.cost = access[n]
nodes_gdf['geometry'] = [Point(xy) for xy in zip(nodes_gdf['x'], nodes_gdf['y'])]
nodes_gdf = gpd.GeoDataFrame(nodes_gdf, crs=crs)

In [101]:
map_5 = KeplerGl(height=600)
map_5.add_data(data=Aimsun_nodes, name = "nodes")
map_5.add_data(data=nodes_gdf, name = "accessibility heatmap")
map_5

User Guide: https://github.com/keplergl/kepler.gl/blob/master/docs/keplergl-jupyter/user-guide.md


KeplerGl(data={'nodes': {'index': [9845, 9848, 9850, 9852, 9854, 9856, 9859, 9863, 9865, 9868, 9881, 9888, 989…

## Previous experiments (might be irrelevant)

In [ ]:
# Bounding box for a small area in East Emeryville, South Berkeley and North Oakland
west, south, east, north = (-122.285535, 37.832531, -122.269571, 37.844596)

# Create a network from that bounding box
G = ox.graph_from_bbox(north, south, east, west, network_type='walk')
# Let's create n arbitrary points of interest (POI)
poi_count = 100

# this function makes those POI into Shapely points
def make_n_pois(north, south, east, west, poi_count):
    for poi in range(poi_count):
        x = (east - west) * random.random() + west
        y = (north - south) * random.random() + south
        yield Point(x, y)
        
pois = list(make_n_pois(north, south, east, west, poi_count))


# Create the plot fig and ax objects but prevent Matplotlib
# from plotting and closing out the plot operation
fig, ax = ox.plot_graph(G, fig_height=10, 
                        show=False, close=False,
                        edge_color='#777777')

# Instead, let's first update the network with these new random POI
for point in pois:
    patch = PolygonPatch(point.buffer(0.0001), fc='#ff0000', ec='k', linewidth=0, alpha=0.5, zorder=-1)
    ax.add_patch(patch)

In [ ]:
def create_nodes_df(G):
    # first make a df from the nodes
    # and pivot the results so that the 
    # individual node ids are listed as
    # row indices
    nodes_df = pd.DataFrame(G.nodes).T
    
    # preserve these indices as a column values, too
    nodes_df['id'] = nodes_df.index
    # and cast it as an integer
    nodes_df['id'] = nodes_df['id'].astype(int)
    
    return nodes_df

nodes_df = create_nodes_df(G)


In [ ]:
nodes_df = pd.DataFrame(G.nodes)

In [ ]:
nodes_df['id'] = nodes_df.index

In [ ]:
nodes_df

In [ ]:
edges_ref = {}

# move through first key (origin node)
for e1 in G.edges.keys():
    e1_dict = G.edges[e1]

    # and then get second key (destination node)
    for e2 in e1_dict.keys():
        # always use the first key here
        e2_dict = e1_dict[e2][0]

        # update the sub-dict to include
        # the origin and destination nodes
        e2_dict['st_node'] = e1
        e2_dict['en_node'] = e2

        # ugly, and unnecessary but might as
        # well name the index something useful
        name = '{}_{}'.format(e1, e2)

        # udpate the top level reference dict
        # with this new, prepared sub-dict
        edges_ref[name] = e2_dict

In [ ]:
[list(G.edges)[0]

In [ ]:
G.edges[list(G.edges)[0]]

In [ ]:
G.edges[list(G.edges)[0]]

In [ ]:
for e1 in G.edges:
    e1_dict = G.edges[e1]
    print(e1_dict)

In [ ]:
# Given a graph, generate a dataframe (df)
# representing all graph edges
def create_edges_df(G):
    # First, we must move the nested objects
    # to a signle top level dictionary
    # that can be consumed by a Pandas df
    edges_ref = {}
    
    # move through first key (origin node)
    for e1 in G.edges.keys():
        e1_dict = G.edges[e1]

        # and then get second key (destination node)
        for e2 in e1_dict.keys():
            # always use the first key here
            e2_dict = e1_dict[e2][0]

            # update the sub-dict to include
            # the origin and destination nodes
            e2_dict['st_node'] = e1
            e2_dict['en_node'] = e2

            # ugly, and unnecessary but might as
            # well name the index something useful
            name = '{}_{}'.format(e1, e2)

            # udpate the top level reference dict
            # with this new, prepared sub-dict
            edges_ref[name] = e2_dict

    # let's take the resulting dict and convert it
    # to a Pandas df, and pivot it as with the nodes
    # method to get unique edges as rows
    edges_df = pd.DataFrame(edges_ref).T
    
    # udpate the edge start and stop nodes as integers
    # which is necessary for Pandana
    edges_df['st_node'] = edges_df['st_node'].astype(int)
    edges_df['en_node'] = edges_df['en_node'].astype(int)
    
    # for the purposes of this example, we are not going
    # to both with impedence along edge so they all get
    # set to the same value of 1
    edges_df['weight'] = 1
    
    return edges_df

edges_df = create_edges_df(G)

In [ ]:
#try some simple plotting of the networkx Graph
H = G.subgraph([n[0] for n in list(G.nodes(data=True))[:10]])
fig, ax = plt.subplots(1, 1, figsize=(10, 10))
nx.draw_networkx(H, ax=ax, node_size=5,
                 font_size=6, alpha=.5,
                 width=.5)
ax.set_axis_off()

In [ ]:
plt.figure(figsize = (10,9))
m = Basemap(projection='merc',llcrnrlon=-180,llcrnrlat=10,urcrnrlon=-50,
urcrnrlat=70, lat_ts=0, resolution='l',suppress_ticks=True)

In [ ]:
#networkx graph visualization in kepler for Fremont
place_name = "Fremont, California"
graph = ox.graph_from_place(place_name, network_type='drive')
fig, ax = ox.plot_graph(graph)

nodes, edges = ox.graph_to_gdfs(graph, nodes=True, edges=True)



In [ ]:
map_3 = KeplerGl(height=600)
map_3.add_data(data=POIs_gdf, name = "Fremont Point of Interests")
map_3.add_data(data=Aimsun_nodes, name = "Fremont Aimsun nodes")
map_3.add_data(data=Aimsun_sections, name = "Fremont edges")
map_3

In [ ]:
from geopy.distance import geodesic 
  
# Loading the lat-long data for Kolkata & Delhi 

# Print the distance calculated in km 



#for (u, v, d) in G.edges(data=True):
    
    

For each selected starting POI, we first find and link it to the closest Aimsun network node. 
Then, by running the BFS, we find the range of reachable spots (accessibility area) from this traversal and count the amount of POIs of different categories reachable within the area. 
And develop an algorithm to calculate the accessibility score
